In [1]:
import warnings
warnings.filterwarnings('ignore')

# tools
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# data processing/sampling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# model/metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import recall_score
from imblearn.metrics import geometric_mean_score

In [2]:
# Load the data
file_path = Path('C:/Users/panka/Documents/Homework/Module_11_Challenge/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [3]:
# Fitting and encoding the columns with the LabelEncoder
le = LabelEncoder()

# Encoding homeowner column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


In [4]:
# Split the Data into Training and Testing

# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [5]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    random_state=1,
    stratify=y)

X_train.shape

(58152, 8)

In [8]:
# Data Pre-Processing

# Create the StandardScaler instance
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Simple Logistic Regression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [14]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



In [16]:
# Oversampling

# Naive Random Oversampling

# Resample the training data with the RandomOversampler
random_oversampler = RandomOverSampler(random_state=1)
X_resampled_1, y_resampled_1 = random_oversampler.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_1)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [17]:
# Train the Logistic Regression model using the resampled data
model_1 = LogisticRegression(solver='lbfgs', random_state=1)
model_1.fit(X_resampled_1, y_resampled_1)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred_1 = model_1.predict(X_test)
accuracy_score_ramdom_oversampler = balanced_accuracy_score(y_test, y_pred_1)
accuracy_score_ramdom_oversampler

0.9948013433551894

In [19]:
# Display the confusion matrix

cm = confusion_matrix(y_test, y_pred_1)

cm_df_os = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df_os

,Predicted 0,Predicted 1
Actual 0,622,3
Actual 1,105,18654


In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_1))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



In [21]:
# SMOTE Oversampling

# Resample the training data with SMOTE
X_resampled_2, y_resampled_2 = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(y_resampled_2)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [22]:
# Train the Logistic Regression model using the resampled data
model_2 = LogisticRegression(solver='lbfgs', random_state=1)
model_2.fit(X_resampled_2, y_resampled_2)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred_2 = model_2.predict(X_test)
balanced_accuracy_score(y_test,y_pred_2)

0.9948279972279972

In [25]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_2)

cm_SMOTE = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_SMOTE

,Predicted 0,Predicted 1
Actual 0,622,3
Actual 1,104,18655


In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



In [28]:
# Undersampling

# Resample the data using the ClusterCentroids resampler
undersample = RandomUnderSampler(random_state=1)
X_resampled_3, y_resampled_3 = undersample.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_3)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [29]:
# Train the Logistic Regression model using the resampled data
model_3 = LogisticRegression(solver='lbfgs', random_state=1)
model_3.fit(X_resampled_3, y_resampled_3)

LogisticRegression(random_state=1)

In [30]:
# Calculate the balanced accuracy score
y_pred_3 = model_3.predict(X_test)
balanced_accuracy_score(y_test, y_pred_3)

0.9948279972279972

In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_3)

cm_undersampler = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_undersampler

,Predicted 0,Predicted 1
Actual 0,622,3
Actual 1,104,18655


In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



In [33]:
# Combination (Over and Under) Sampling

# Resample the training data with SMOTEENN
combination = SMOTEENN(random_state=1)
X_resampled_4, y_resampled_4 = combination.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_4)

Counter({'high_risk': 55299, 'low_risk': 55918})

In [34]:
# Train the Logistic Regression model using the resampled data
model_4 = LogisticRegression(solver='lbfgs', random_state=1)
model_4.fit(X_resampled_4, y_resampled_4)

LogisticRegression(random_state=1)

In [35]:
# Calculate the balanced accuracy score
y_pred_4 = model_4.predict(X_test)
balanced_accuracy_score(y_test, y_pred_4)

0.994748035609574

In [36]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_4)

cm_combination = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_combination

,Predicted 0,Predicted 1
Actual 0,622,3
Actual 1,107,18652


In [37]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



In [38]:
# Final Questions

#1. Which model had the best balanced accuracy score?

   # The random oversampler, SMOTE, undersampler and combination had the best balanced accuracy score of 0.9948

#2. Which model had the best recall score?

   # All the model had a recall score of 0.99

#3. Which model had the best geometric mean score?

    # All the model had a geometric mean score of 0.99.
